Upload data to huggingface dataset

In [1]:
import pandas as pd
import os
import json
from datasets import DatasetDict
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from datasets import load_dataset, DatasetDict

In [2]:
file_path = "/home/ubuntu/multimodal/Dataset/Climate-raw/temp.csv"

df = pd.read_csv(file_path)

df['fut_temp'] = df['temp'].shift(-1)
df['fut_summary'] = df['text'].shift(-1)
df['fut_date'] = df['date'].shift(-1)

df['summary_temp'] = df.apply(lambda x: json.dumps({"date": x['date'], "summary": x['text'], "temperature": x['temp']}), axis=1)
df['fut_summary_temp'] = df.apply(lambda x: json.dumps({"date": x['fut_date'], "summary": x['fut_summary'], "temperature": x['fut_temp']}), axis=1)
df['temp'] = df.apply(lambda x: json.dumps({"date": x['date'], "temperature": x['temp']}), axis=1)
df['fut_temp'] = df.apply(lambda x: json.dumps({"date": x['fut_date'], "temperature": x['fut_temp']}), axis=1)
df['summary'] = df.apply(lambda x: json.dumps({"date": x['date'], "summary": x['text']}), axis=1)
df['fut_summary'] = df.apply(lambda x: json.dumps({"date": x['fut_date'], "summary": x['fut_summary']}), axis=1)

df.to_csv('/home/ubuntu/multimodal/Dataset/Climate/temp.csv', index=False)


         date                                               text  \
0  2014-04-04  ...Metadata...\nEXTENDED FORECAST DISCUSSION\n...   
1  2014-04-05  ...Metadata...\nEXTENDED FORECAST DISCUSSION\n...   
2  2014-04-06  ...Metadata...\nEXTENDED FORECAST DISCUSSION\n...   
3  2014-04-07  ...Metadata...\nEXTENDED FORECAST DISCUSSION\n...   
4  2014-04-08  ...Metadata...\nEXTENDED FORECAST DISCUSSION\n...   

                                          temp  \
0  {"date": "2014-04-04", "temperature": 54.9}   
1  {"date": "2014-04-05", "temperature": 53.1}   
2  {"date": "2014-04-06", "temperature": 49.5}   
3  {"date": "2014-04-07", "temperature": 48.4}   
4  {"date": "2014-04-08", "temperature": 58.3}   

                                      fut_temp  \
0  {"date": "2014-04-05", "temperature": 53.1}   
1  {"date": "2014-04-06", "temperature": 49.5}   
2  {"date": "2014-04-07", "temperature": 48.4}   
3  {"date": "2014-04-08", "temperature": 58.3}   
4  {"date": "2014-04-09", "temperature":

In [3]:
def split_data(file_path, train_ratio=0.8, validation_ratio=0.1, test_ratio=0.1):
    # Read the CSV file
    data = pd.read_csv(file_path)
    
    # Calculate the validation and test sizes
    val_size = validation_ratio / (test_ratio + validation_ratio)
    
    # Split the data into train and temporary datasets
    train_data, temp_data = train_test_split(data, test_size=(1 - train_ratio), random_state=42, shuffle=False)
    
    # Split the temporary dataset into validation and test datasets
    validation_data, test_data = train_test_split(temp_data, test_size=val_size, random_state=42, shuffle=False)
    
    # Save the datasets
    
    dir = file_path.split('/')[:-1]
    dir = '/'.join(dir)
    file_name = file_path.split('/')[-1]
    train_data.to_csv(f'{dir}/train_{file_name}', index=False)
    validation_data.to_csv(f'{dir}/val_{file_name}', index=False)
    test_data.to_csv(f'{dir}/test_{file_name}', index=False)

# List of your CSV files
dir = '/home/ubuntu/multimodal/Dataset/Climate'

# Loop through each file and split the data
for filename in os.listdir(dir):
    if not filename.startswith("train") and not filename.startswith("test") and not filename.startswith("val"):
        path = os.path.join(dir, filename)
        split_data(path)  # Adjust the directory path as needed

In [4]:
def convert_to_parquet(dataframe_test, dataframe_train, dataframe_val):
    train = pd.concat(dataframe_train)
    test = pd.concat(dataframe_test)
    val = pd.concat(dataframe_val)

    train_path = '../parquet_dir/train_finance.parquet'
    test_path = '../parquet_dir/test_finance.parquet'
    val_path = '../parquet_dir/val_finance.parquet'

    train.to_parquet(train_path, engine='pyarrow')
    test.to_parquet(test_path, engine='pyarrow')
    val.to_parquet(val_path, engine='pyarrow')
    # Load the dataset
    train_dataset = load_dataset('parquet', data_files=train_path, split='train')
    test_dataset = load_dataset('parquet', data_files=test_path, split='train')
    val_dataset = load_dataset('parquet', data_files=val_path, split = 'train')
    dataset_dict = DatasetDict({
        'train': train_dataset,
        'validation': val_dataset,
        'test': test_dataset
    })

    return dataset_dict

In [13]:
def load_from_huggingface(dataset_path, case, units, dataset_name):
    dataset = load_dataset(dataset_path)

    if not os.path.exists(f"../Data/{dataset_name}/{units}/{case}"):
        os.makedirs(f"../Data/{dataset_name}/{units}/{case}")

# Access the train split
    for split in ['train', 'validation', 'test']:
        train_dataset = dataset[split]

        # Convert the dataset to a Pandas DataFrame
        df = train_dataset.to_pandas()
        # Save the DataFrame to a CSV file
        df.to_csv(f"../Data/{dataset_name}/{units}/{case}/{split}_all.csv", index=False)

In [7]:
def combine_window(df, window_size, unit):
    json_data = []
    end_index = len(df) - 2 * window_size + 1


    for i in range(end_index):
        combined_input = {}
        combine_output = {}
        
        for j in range(window_size):
            input_key = f"{unit}_{j+1}"
            output_key = f"{unit}_{j+window_size+1}"
            combined_input[input_key] = json.loads(df.iloc[i + j]['input'])
            combine_output[output_key] = json.loads(df.iloc[i + j + window_size - 1]['output'])
        combine_json = {
            "input": json.dumps(combined_input),
            "output": json.dumps(combine_output),
            "instruction": df.iloc[i]['instruction']
        }
        json_data.append(combine_json)
    
    json_df = pd.DataFrame(json_data)
    return json_df

# Case 3 text + number  => text + number

In [32]:
dir = '../Dataset/Climate'
dataframe_train = []
dataframe_test = []
dataframe_val = []
window_size = 3

def create_mixed_mixed(filename, window_size, unit):
    if filename.startswith('train') or filename.startswith('test') or filename.startswith('val'):
        path = os.path.join(dir, filename)
        summaries = pd.read_csv(path)

        if window_size == 1:
            window = f"{window_size} {unit}"
        else:
            window = f"{window_size} {unit}s"
        example_output = {}
        for i in range(window_size):
            example_output[f"{unit}_{i+1+window_size}"] = {"date": "<date>", "summary": "<summary>", "temprature": "<temprature>"}
        example_output = json.dumps(example_output)
        instruction = f"Given the summary and the temprature for {window}, please predict the next {window}'s temprature and summary in json format. And ouput only the json data, the response should only contain {example_output}"
        
        df = summaries[['summary_temp', 'fut_summary_temp']].rename(columns={'summary_temp': 'input', 'fut_summary_temp': 'output'})
        df['instruction'] = instruction
        # skip the first and last historical_size days
        return combine_window(df, window_size, "day")

for filename in os.listdir(dir):
    df = create_mixed_mixed(filename, window_size, "day")
    if filename.startswith('test'):
        dataframe_test.append(df)
    elif filename.startswith('train'):
        dataframe_train.append(df)
    elif filename.startswith('val'):
        dataframe_val.append(df)

        
dataset_dict = convert_to_parquet(dataframe_test, dataframe_train, dataframe_val)
token = os.getenv("HF_TOKEN")
# Push the dataset to the Hugging Face Hub
dataset_dict.push_to_hub(f"Howard881010/climate-{window_size}_day-mixed-mixed", token=token)

# Load the dataset from Hugging Face Hub
load_from_huggingface(f"Howard881010/climate-{window_size}_day-mixed-mixed", "mixed-mixed", f"{window_size}_day", "Climate")

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/567 [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2818 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/348 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/348 [00:00<?, ? examples/s]

# Case 4 text => text


In [31]:
dir = '../Dataset/Climate'
dataframe_train = []
dataframe_test = []
dataframe_val = []
window_size = 3

def create_text_text(filename, window_size, unit):
    if filename.startswith('train') or filename.startswith('test') or filename.startswith('val'):
        path = os.path.join(dir, filename)
        summaries = pd.read_csv(path)

        if window_size == 1:
            window = f"{window_size} {unit}"
        else:
            window = f"{window_size} {unit}s"
        example_output = {}
        for i in range(window_size):
            example_output[f"{unit}_{i+1+window_size}"] = {"date": "<date>", "summary": "<summary>"}
        example_output = json.dumps(example_output)
        instruction = f"Given the summary for {window}, please predict the next {window}'s summary in json format. And ouput only the json data, the response should only contain {example_output}"
        
        df = summaries[['summary', 'fut_summary']].rename(columns={'summary': 'input', 'fut_summary': 'output'})
        df['instruction'] = instruction
        # skip the first and last historical_size days
        return combine_window(df, window_size, "day")

for filename in os.listdir(dir):
    df = create_text_text(filename, window_size, "day")
    if filename.startswith('test'):
        dataframe_test.append(df)
    elif filename.startswith('train'):
        dataframe_train.append(df)
    elif filename.startswith('val'):
        dataframe_val.append(df)
        
dataset_dict = convert_to_parquet(dataframe_test, dataframe_train, dataframe_val)
token = os.getenv("HF_TOKEN")
# Push the dataset to the Hugging Face Hub
dataset_dict.push_to_hub(f"Howard881010/climate-{window_size}_day-text-text", token=token)

# Load the dataset from Hugging Face Hub
load_from_huggingface(f"Howard881010/climate-{window_size}_day-text-text", "text-text", f"{window_size}_day", "Climate")

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Generating train split:   0%|          | 0/2818 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/348 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/348 [00:00<?, ? examples/s]

# Put all data together

In [33]:
dir = '../Dataset/Climate'
dataframe_train = []
dataframe_test = []
dataframe_val = []
window_size = 3


for filename in os.listdir(dir):
    for size in range(1, window_size+1):
        df1 = create_text_text(filename, size, "day")
        df2 = create_mixed_mixed(filename, size, "day")
        if filename.startswith('test'):
            dataframe_test.append(df1)
            dataframe_test.append(df2)
        elif filename.startswith('train'):
            dataframe_train.append(df1)
            dataframe_train.append(df2)
        elif filename.startswith('val'):
            dataframe_val.append(df1)
            dataframe_val.append(df2)
        
dataset_dict = convert_to_parquet(dataframe_test, dataframe_train, dataframe_val)
token = os.getenv("HF_TOKEN")
# Push the dataset to the Hugging Face Hub
dataset_dict.push_to_hub(f"Howard881010/climate", token=token)


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/17 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Howard881010/climate/commit/162186f448bc4a5a734ef15afb70d30489c9802d', commit_message='Upload dataset', commit_description='', oid='162186f448bc4a5a734ef15afb70d30489c9802d', pr_url=None, pr_revision=None, pr_num=None)